In [1]:
from abc import ABC, abstractmethod, ABCMeta
import datetime
from typing import Any, Optional, List, Callable
from functools import wraps
import json
import logging


class OrderLogger:
    """Класс для настройки и управления системой логирования."""
    def __init__(self, log_file: str = 'orders.log'):
        """
        Инициализирует систему логирования.
        Args:
            log_file (str): Путь к файлу для записи логов. По умолчанию 'orders.log'.
        """
        self.logger = logging.getLogger('OrderSystem')
        self.logger.setLevel(logging.DEBUG)
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(formatter)
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        self.logger.addHandler(console_handler)
    def log_info(self, message: str):
        """Записывает информационное сообщение в лог.
        Args:
            message (str): Сообщение для логирования.
        """
        self.logger.info(message)
    def log_warning(self, message: str):
        """Записывает предупреждение в лог.
        Args:
            message (str): Сообщение для логирования.
        """
        self.logger.warning(message)
    def log_error(self, message: str):
        """Записывает сообщение об ошибке в лог.

        Args:
            message (str): Сообщение для логирования.
        """
        self.logger.error(message)
    def log_debug(self, message: str):
        """Записывает отладочное сообщение в лог.
        Args:
            message (str): Сообщение для логирования.
        """



class OrderError(Exception):
    """Абстрактный класс исключений"""
    pass



class InvalidOrderError(OrderError):
    """Исключение: некорректные данные заказа"""
    def __init__(self, message: str = "Некорректные данные заказа", errors: dict = None):
        """Инициализация исключения"""
        super().__init__(message)
        self.errors = errors or {}

class PermissionDeniedError(OrderError):
    """Исключение: доступ запрещён"""
    def __init__(self, user: str = None, action: str = None):
        """Инициализация исключения"""
        message = f"Доступ запрещен"
        if user and action:
            message = f"Пользователь {user} не имеет прав для выполнения действия: {action}"
        super().__init__(message)
        self.user = user
        self.action = action

class OrderNotFoundError(OrderError):
    """Исключение: заказ не найден"""
    def __init__(self, order_id: str = None):
        """Инициализация исключения"""
        message = "Заказ не найден"
        if order_id:
            message = f"Заказ {order_id} не найден"
        super().__init__(message)
        self.order_id = order_id



class OrderMeta(type):
    """Метакласс для заказов"""
    _registry = {}
    def __new__(cls, name, bases, namespace):
        """Инициализация"""
        new_class = super().__new__(cls, name, bases, namespace)
        if bases and bases[0].__name__ == 'Order':
            cls._registry[name.lower()] = new_class
        return new_class

class CombinedMeta(OrderMeta, ABCMeta):
    """Абстрактный метакласс для заказов"""
    pass



class LoggingMixin:
    """Миксин для логгирования"""
    def log_action(self, action: str):
        """
        Вывод сообщения:
        Args:
            action: действие
        """
        message = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, Заказ №{self._order_id} обновлен: {action}"
        print(message)

class NotificationMixin:
    """Миксин для отправки уведомления"""
    def send_notification(self, message: str):
        """
        Вывод сообщения:
        Args:
            message: сообщение
        """
        notification = f"Уведомление для клиента {self._customer}: {message}"
        print(notification)



class Trackable(ABC):
    """Интерфейс для вывода статуса"""
    @abstractmethod
    def track_status(self):
        pass

class Reportable(ABC):
    """Интерфейс для вывода счета"""
    @abstractmethod
    def generate_invoice(self):
        pass



class Order(ABC, LoggingMixin, NotificationMixin, metaclass=CombinedMeta):
    """Абстрактный базовый класс для заказов."""
    def __init__(self, order_id: int, customer: str, order_date: datetime, status: str, items: list[dict[str: Any]], logger: OrderLogger):
        """
        Инициализирует заказ.

        Args:
            order_id (int): Уникальный идентификатор заказа.
            customer (str): Имя клиента.
            order_date (datetime): Дата создания заказа.
            status (str): Текущий статус заказа.
            items (List[Dict[str, Any]]): Список товаров в заказе.
            logger (OrderLogger): Экземпляр логгера.
        """
        self._order_id = order_id
        self._customer = customer
        self._order_date = order_date
        self._status = status
        self._items = items.copy()
        self.logger = logger
        self.logger.log_info(f'Создан заказ №{order_id} для клиента {customer}')
    @classmethod
    def from_dict(cls, data: dict[str: Any], logger: OrderLogger):
        """
        Создает объект Order из словаря.
        :param data:
        :param logger:
        :return:
        """
        logger.log_debug(f'Десериализация заказа №{data.get("order_id")}')
        order_class = {
            'OnlineOrder': OnlineOrder,
            'PhoneOrder': PhoneOrder,
            'StoreOrder': StoreOrder
        }.get(data['type'])
        if not order_class:
            raise ValueError(f"Неизвестный тип заказа: {data['type']}")
        return order_class._create_from_dict(data, logger)
    @classmethod
    @abstractmethod
    def _create_from_dict(cls, data: dict[str: Any]):
        """
        Создает конкретный тип заказа из словаря.
        :param data:
        :return:
        """
        pass
    def to_dict(self):
        """
        Преобразует заказ в словарь
        :return dict:
        """
        self.logger.log_debug(f'Преобразование заказа №{self._order_id} в словарь')
        result = {
            'type': self.__class__.__name__,
            'order_id': self._order_id,
            'customer': self._customer,
            'order_date': self._order_date.strftime('%Y-%m-%d %H:%M:%S'),
            'status': self._status,
            'items': self._items
        }
        return result
    @property
    def order_id(self):
        """
        Возвращает order_id
        :return: self._order_id
        """
        return self._order_id
    @property
    def customer(self):
        """
        Возвращает customer
        :return: self._customer
        """
        return self._customer
    @property
    def order_date(self):
        """
        self._order_date order_date
        :return: self._order_date
        """
        return self._order_date
    @property
    def status(self):
        """
        Возвращает status
        :return: self._status
        """
        return self._status
    @property
    def items(self):
        """
        Возвращает items
        :return: self._items
        """
        return self._items
    @order_id.setter
    def order_id(self, value: int):
        """
        Сеттер для order_id
        :param value:
        :return:
        """
        self._order_id = value
    @customer.setter
    def customer(self, value: str):
        """
        Сеттер для customer
        :param value:
        :return:
        """
        self._customer = value
    @order_date.setter
    def order_date(self, value: datetime):
        """
        Сеттер для order_date
        :param value:
        :return:
        """
        self._order_date = value
    @status.setter
    def status(self, value: str):
        """
        Сеттер для status
        :param value:
        :return:
        """
        action = f'Статус изменён с {self._status} на '
        self._status = value
        action += self._status
        self.log_action(action)
        if self._status == 'доставлен':
            message = f'Заказ №{self._order_id} доставлен!'
            self.send_notification(message)
        elif self._status == 'отменен':
            message = f'Заказ №{self._order_id} отменен!'
            self.send_notification(message)
    @items.setter
    def items(self, value: list[dict[str: Any]]):
        """
        Сеттер для items
        :param value:
        :return:
        """
        action = f'Изменен список товаров с {self._items} на {value}'
        self._items = value.copy()
        self.log_action(action)
    @abstractmethod
    def calculate_total(self):
        """
        Абстрактный метод для подсчета стоимости заказа
        :return:
        """
        pass
    def __str__(self):
        """
        Преобразование заказа в строку
        :return: f'Заказ №: {self._order_id}. Статус: {self.status}.'
        """
        return f'Заказ №: {self._order_id}. Статус: {self.status}.'
    def __lt__(self, other: 'Order'):
        """
        Сравнение заказов по стоимости
        :param other:
        :return:
        """
        if not isinstance(other, Order):
            raise TypeError('Сравнение возможно только между объектами Order.')
        else:
            return self.calculate_total() < other.calculate_total()
    def __gt__(self, other: 'Order'):
        """
        Сравнение заказов по дате
        :param other:
        :return:
        """
        if not isinstance(other, Order):
             raise TypeError('Сравнение возможно только между объектами Order.')
        else:
            return self._order_date > other._order_date
    def __eq__(self, other: 'Order'):
        """
        Проверяет равна ли стоимость self стоимости other
        :param other:
        :return:
        """
        return self.calculate_total() == other.calculate_total()

class OnlineOrder(Order, Trackable, Reportable):
    """Класс для онлайн-заказов."""
    def __init__(self, order_id: int, customer: str, order_date: datetime, status: str, items: list[dict[str: Any]], payment_method: str, logger: OrderLogger):
        """
        Инициализирует онлайн-заказ.

        Args:
            order_id (int): Уникальный идентификатор заказа.
            customer (str): Имя клиента.
            order_date (datetime): Дата создания заказа.
            status (str): Текущий статус заказа.
            items (List[Dict[str, Any]]): Список товаров в заказе.
            payment_method (str): Способ оплаты.
            logger (OrderLogger): Экземпляр логгера.
        """
        super().__init__(order_id, customer, order_date, status, items, logger)
        self._payment_method = payment_method
    def to_dict(self):
        """
        Преобразование заказа в словарь
        :return: dict
        """
        result = super().to_dict()
        result['payment_method'] = self._payment_method
        return result
    @classmethod
    def _create_from_dict(cls, data: dict[str: Any], logger: OrderLogger):
        """
        Преобразование словаря в заказ
        :param data: словарь
        :return:
        """
        return cls(
            order_id=data['order_id'],
            customer=data['customer'],
            order_date=datetime.datetime.fromisoformat(data['order_date']),
            status=data['status'],
            items=data['items'],
            payment_method=data['payment_method'],
            logger=logger
        )
    def calculate_total(self):
        """
        Подсчет стоимости заказа
        :return:
        """
        res = 0
        for i in self._items:
            j = list(i.values())
            res += j[0] * j[1]
        return res
    def __str__(self):
        """
        Преобразование в строку
        :return:
        """
        return f'Онлайн-заказ №: {self._order_id}. Статус: {self.status}. Способ оплаты: {self._payment_method}.'
    @property
    def payment_method(self):
        """
        Возвращает payment_method
        :return:
        """
        return self._payment_method
    @payment_method.setter
    def payment_method(self, val):
        """
        Сеттер для payment_method
        :param val:
        :return:
        """
        self._payment_method = val
    def track_status(self):
        """
        Возвращает статус и ожидаемую даты доставки
        :return:
        """
        delivery_date = self.order_date + datetime.timedelta(days=3)
        return f"Статус: {self._status}, Ожидаемая дата доставки: {delivery_date.strftime('%d.%m.%Y')}"
    def generate_invoice(self):
        """
        Возвращает счет
        :return:
        """
        return f"Счет для онлайн-заказа №{self._order_id}: {self.calculate_total():.2f} руб."

class PhoneOrder(Order, Trackable, Reportable):
    """Класс для заказов по телефону"""
    def __init__(self, order_id: int, customer: str, order_date: datetime, status: str, items: list[dict[str: Any]], operator_name: str, logger: OrderLogger):
        """
        Инициализирует заказа по телефону.

        Args:
            order_id (int): Уникальный идентификатор заказа.
            customer (str): Имя клиента.
            order_date (datetime): Дата создания заказа.
            status (str): Текущий статус заказа.
            items (List[Dict[str, Any]]): Список товаров в заказе.
            operator_name (str): Оператор.
            logger (OrderLogger): Экземпляр логгера.
        """
        super().__init__(order_id, customer, order_date, status, items, logger)
        self._operator_name = operator_name
        self.logger.log_info(f'Создан онлайн-заказ №{order_id}, оператор: {operator_name}')
    @classmethod
    def _create_from_dict(cls, data: dict[str: Any], logger: OrderLogger):
        """Создание заказк из словаря"""
        return cls(
            order_id=data['order_id'],
            customer=data['customer'],
            order_date=datetime.datetime.fromisoformat(data['order_date']),
            status=data['status'],
            items=data['items'],
            operator_name=data['operator_name'],
            logger=logger
        )
    def to_dict(self):
        """
        Преобразование закза в словарь
        :return:
        """
        result = super().to_dict()
        result['operator_name'] = self._operator_name
        return result
    def calculate_total(self):
        """
        Подсчет стоимости
        :return:
        """
        res = 0
        for i in self._items:
            j = list(i.values())
            res += j[0] * j[1]
        return res
    def __str__(self):
        """
        Преобразует в строку
        :return:
        """
        return f'Заказ по телефону №: {self._order_id}. Статус: {self.status}. Оператор: {self._operator_name}.'
    @property
    def operator_name(self):
        """
        Возвращает operator_name
        :return:
        """
        return self._operator_name
    @operator_name.setter
    def operator_name(self, val):
        """
        Сеттер для operator_name
        :param val:
        :return:
        """
        self._operator_name = val
    def track_status(self):
        """
        Возвращает статус
        :return:
        """
        return f"Статус: {self._status}, Оператор: {self._operator_name}"
    def generate_invoice(self):
        """
        Возвращает счет
        :return:
        """
        return f"Счет для онлайн-заказа №{self._order_id}: {self.calculate_total():.2f} руб."

class StoreOrder(Order, Trackable, Reportable):
    """Класс для заказов в магазине"""
    def __init__(self, order_id: int, customer: str, order_date: datetime, status: str, items: list[dict[str: Any]], store_location: str, logger: OrderLogger):
        """
        Инициализирует заказ в магазине.

        Args:
            order_id (int): Уникальный идентификатор заказа.
            customer (str): Имя клиента.
            order_date (datetime): Дата создания заказа.
            status (str): Текущий статус заказа.
            items (List[Dict[str, Any]]): Список товаров в заказе.
            store_location (str): Расположение магазина.
            logger (OrderLogger): Экземпляр логгера.
        """
        super().__init__(order_id, customer, order_date, status, items, logger)
        self._store_location = store_location
        self.logger.log_info(f'Создан онлайн-заказ №{order_id}, местонахождение магазина: {store_location}')
    @classmethod
    def _create_from_dict(cls, data: dict[str: Any], logger: OrderLogger):
        """
        Преобразование словаря в заказ
        :param data:
        :return:
        """
        return cls(
            order_id=data['order_id'],
            customer=data['customer'],
            order_date=datetime.datetime.fromisoformat(data['order_date']),
            status=data['status'],
            items=data['items'],
            store_location=data['store_location'],
            logger=logger
        )
    def to_dict(self):
        """
        Преобразует заказ в словарь
        :return:
        """
        result = super().to_dict()
        result['store_location'] = self._store_location
        return result
    def calculate_total(self):
        """
        Подсчет стоимости
        :return:
        """
        res = 0
        for i in self._items:
            j = list(i.values())
            res += j[0] * j[1]
        return res
    def __str__(self):
        """Преобразование в строку"""
        return f'Заказ в магазине №: {self._order_id}. Статус: {self.status}. Местонахождение магазина: {self._store_location}.'
    @property
    def store_location(self):
        """
        Возвращает местонахождение магазина
        :return:
        """
        return self._store_location
    @store_location.setter
    def store_location(self, val):
        """
        Сеттер для store_location
        :param val:
        :return:
        """
        self._store_location = val
    def track_status(self):
        """
        Возвращает статус
        :return:
        """
        return f"Статус: {self._status}, Магазин: {self._store_location}"
    def generate_invoice(self):
        """
        Возвращает счет
        :return:
        """
        return f"Счет для онлайн-заказа №{self._order_id}: {self.calculate_total():.2f} руб."



class Address:
    """Класс для адреса"""
    def __init__(self, city: str, street: str, building: str, apartment: Optional[str] = None):
        """
        Инициализация
        :param city: город
        :param street: улича
        :param building: дом
        :param apartment: квартира
        """
        self.city = city
        self.street = street
        self.building = building
        self.apartment = apartment
    def __str__(self):
        """
        Преобразует в строку
        :return:
        """
        res = f'Город: {self.city}, улица: {self.street},  дом: {self.building}'
        if self.apartment:
            res += f', квартира: {self.apartment}'
        return res

class Customer:
    """Класс клиент"""
    def __init__(self, name: str, email: str, phone: str, address: Address):
        """
        Иициализация
        :param name: ФИО
        :param email: адрес электронной почты
        :param phone: телефон
        :param address: адрес
        """
        self._name = name
        self._email = email
        self._phone = phone
        self._address = address
        self._order_history: List['Order'] = []
    def place_order(self, order: 'Order'):
        """
        Добавление заказа
        :param order: заказ
        :return:
        """
        self._order_history.append(order)
    def get_order_history(self):
        """
        Возвращает историю заказов
        :return:
        """
        return self._order_history
    def set_shipping_address(self, address: Address):
        """
        Меняет адрес
        :param address:
        :return:
        """
        self._address = address
    @property
    def name(self):
        """Возвращает ФИО"""
        return self._name
    @name.setter
    def name(self, val):
        """Сеттер для name"""
        self._name = val
    @property
    def email(self):
        """Возвращает адрес электронной почты"""
        return self._email
    @email.setter
    def email(self, val):
        """Сеттер для email"""
        self._email = val
    @property
    def phone(self):
        """Возвращает номер телефона"""
        return self._phone
    @phone.setter
    def phone(self, val):
        """Сеттер для phone"""
        self._phone = val
    @property
    def address(self):
        """Возвращает адрес"""
        return self._address



class OrderFactory:
    """Класс для фабричного метода"""
    @classmethod
    def create_order(cls, order_type: str, **kwargs):
        """
        Создание заказа
        :param order_type: тип заказа
        :param kwargs:
        :return:
        """
        if order_type == 'online':
            return OnlineOrder(**kwargs)
        elif order_type == 'phone':
            return PhoneOrder(**kwargs)
        elif order_type == 'store':
            return StoreOrder(**kwargs)
        else:
            raise ValueError



class OrderCancelHandler(ABC):
    """Класс для отмены заказа"""
    def __init__(self, successor: Optional['OrderCancelHandler'] = None):
        self._successor = successor
    @abstractmethod
    def handle_request(self, order: 'Order', reason: str):
        pass
    def _pass_to_next(self, order: 'Order', reason: str):
        if self._successor:
            return self._successor.handle_request(order, reason)
        return False

class CallCenterOperator(OrderCancelHandler):
    """Обработка отмены в колл-центре"""
    def handle_request(self, order: 'Order', reason: str):
        if not order.status in ["отправлен", "доставлен"]:
            print(f"Оператор одобрил отмену заказа №{order.order_id}. Причина: {reason}")
            order.status = "отменен"
            order.log_action(f"Отменен оператором. Причина: {reason}")
            return True
        return self._pass_to_next(order, reason)

class Manager(OrderCancelHandler):
    """Обработка отмены менеджером"""
    def handle_request(self, order: 'Order', reason: str) -> bool:
        if not order.status == 'доставлен':
            print(f"Менеджер одобрил отмену заказа №{order.order_id}. Причина: {reason}")
            order.status = "отменен"
            order.log_action(f"Отменен менеджером. Причина: {reason}")
            return True
        return self._pass_to_next(order, reason)

class Admin(OrderCancelHandler):
    """Обработка отмены администратором"""
    def handle_request(self, order: 'Order', reason: str) -> bool:
        print(f"Администратор одобрил отмену заказа №{order.order_id}. Причина: {reason}")
        order.status = "отменен"
        order.log_action(f"Отменен администратором. Причина: {reason}")
        return True

class OrderCancelProcessor:
    """Класс для отмены заказа"""
    def __init__(self):
        self._handler_chain = CallCenterOperator(
            Manager(
                Admin()
            )
        )
    def process_cancellation(self, order: 'Order', reason: str) -> bool:
        print(f"\nОбработка запроса на отмену заказа №{order.order_id}...")
        print(f"Текущий статус: {order.status}")
        return self._handler_chain.handle_request(order, reason)



class TotalCalculator(ABC):
    """Абстрактный класс для подсчета стоимости заказа"""
    def calculate_subtotal(self, items: list[dict[str: Any]]):
        """
        Подсчет стоимости
        :param items:
        :return:
        """
        subtotal = 0.0
        for i in items:
            j = list(i.values())
            subtotal += j[0] * j[1]
        return subtotal
    @abstractmethod
    def calculate_discount(self, subtotal: float):
        """Подсчет скидок"""
        pass
    @abstractmethod
    def calculate_additional_fees(self, subtotal: float):
        """Подсчет дополнительных сборов"""
        pass
    def calculate_total(self, items: list[dict[str: Any]]):
        """
        Подсчет общей стоимости
        :param items:
        :return:
        """
        subtotal = self.calculate_subtotal(items)
        discount = self.calculate_discount(subtotal)
        fees = self.calculate_additional_fees(subtotal)
        total = subtotal - discount + fees
        return total

class OnlineOrderCalculator(TotalCalculator):
    """Подсчет стоимости для онлайн-заказа"""
    def __init__(self, payment_method: str):
        self.payment_method = payment_method
    def calculate_discount(self, subtotal: float):
        if self.payment_method == 'online':
            return subtotal * 0.05
        return 0.0
    def calculate_additional_fees(self, subtotal: float):
        return 0.0

class PhoneOrderCalculator(TotalCalculator):
    """Подсчет стоимости для заказа по телефону"""
    def calculate_discount(self, subtotal: float):
        return 0.0
    def calculate_additional_fees(self, subtotal: float):
        return 10.0

class StoreOrderCalculator(TotalCalculator):
    """Подсчет стоимости для заказа в магазине"""
    def __int__(self, store_location: str):
        self.store_location = store_location
    def calculate_discount(self, subtotal: float):
        if self.store_location in ['Москва', 'Санкт-Петербург', 'Казань']:
            return subtotal * 0.1
        elif self.store_location in ['Новосибирск', 'Хабаровск', 'Владивосток']:
            return subtotal * 0.5
        return 0.0
    def calculate_additional_fees(self, subtotal: float):
        return 0.0



class OrderRepository:
    """Класс для работы с хранением заказов."""
    def __init__(self, logger: OrderLogger):
        """
        Инициализирует репозиторий заказов.
        Args:
            logger (OrderLogger): Экземпляр логгера.
        """
        self.logger = logger
    def save_to_file(self, orders: List[Order], filename: str) -> None:
        """Сохраняет список заказов в JSON-файл.
        Args:
            orders (List[Order]): Список заказов для сохранения.
            filename (str): Имя файла для сохранения.
        Raises:
            IOError: Если возникла ошибка при записи в файл.
        """
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump([order.to_dict() for order in orders], f, indent=2, ensure_ascii=False)
            self.logger.log_info(f'Успешно сохранено {len(orders)} заказов в файл {filename}')
        except IOError as e:
            self.logger.log_error(f'Ошибка при сохранении заказов: {str(e)}')
            raise

    def load_from_file(self, filename: str) -> List[Order]:
        """Загружает список заказов из JSON-файла.
        Args:
            filename (str): Имя файла для загрузки.
        Returns:
            List[Order]: Список загруженных заказов.
        Raises:
            IOError: Если возникла ошибка при чтении файла.
            ValueError: Если данные в файле некорректны.
        """
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
            orders = [Order.from_dict(order_data, self.logger) for order_data in data]
            self.logger.log_info(f'Успешно загружено {len(orders)} заказов из файла {filename}')
            return orders
        except IOError as e:
            self.logger.log_error(f'Ошибка при загрузке заказов: {str(e)}')
            raise
        except json.JSONDecodeError as e:
            error_msg = f'Некорректный JSON в файле {filename}: {str(e)}'
            self.logger.log_error(error_msg)
            raise ValueError(error_msg)



def check_permissions(required_roles: List[str]):
    def decorator(func: Callable):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if not args[2].role in required_roles:
                raise PermissionDeniedError(user=args[2].name, action=func.__name__)
            return func(*args, **kwargs)
        return wrapper
    return decorator

class User:
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role

class OrderService:
    @check_permissions(required_roles=['admin', 'manager'])
    def cancel_order(self, order_id: str, user: User):
        print(f"Заказ {order_id} отменен пользователем {user.name}")
    @check_permissions(required_roles=['admin'])
    def change_status(self, order_id: str, new_status: str, user: User):
        print(f"Статус заказа {order_id} изменен на {new_status} пользователем {user.name}")



def main():
    logger = OrderLogger()
    adr = Address(
        city='Москва',
        street='Ленина',
        building='11'
    )
    cus = Customer(
        name='Иван Иванов',
        email='ivanivano@main.ru',
        phone='+79666941731',
        address=adr
    )
    order = OrderFactory.create_order(
        'store',
            order_id=3,
            customer="Иван Иванов",
            order_date=datetime.datetime.now(),
            status="готовится",
            items=[{"name": "Наушники", "price": 5000, "quantity": 1}],
            store_location="Москва, Тверская, 10",
        logger=logger
    )
    cus.place_order(order)
    for i in cus.get_order_history():
        print(i.track_status())
        i.status = 'готов'
    for i in cus.get_order_history():
        print(i.track_status())




if __name__ == '__main__':
    main()

2025-04-25 23:55:38 - OrderSystem - INFO - Создан заказ №3 для клиента Иван Иванов
INFO:OrderSystem:Создан заказ №3 для клиента Иван Иванов
2025-04-25 23:55:38 - OrderSystem - INFO - Создан онлайн-заказ №3, местонахождение магазина: Москва, Тверская, 10
INFO:OrderSystem:Создан онлайн-заказ №3, местонахождение магазина: Москва, Тверская, 10


Статус: готовится, Магазин: Москва, Тверская, 10
2025-04-25 23:55:38, Заказ №3 обновлен: Статус изменён с готовится на готов
Статус: готов, Магазин: Москва, Тверская, 10
